In [8]:
import ipyleaflet
import ogr, os
import pandas as pd
import geopandas as gpd
import shapefile
import json 
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl, basemaps, basemap_to_tiles
)

import pyproj
from shapely.ops import transform
from shapely.geometry import Polygon
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

from IPython.html import widgets
from IPython.display import display
import osmnx as ox
from tkinter import filedialog
from tkinter import *
root = Tk()
root.withdraw()
out = filedialog.askdirectory()


geo={'Open Street Maps':['North Carolina','South Carolina'],'MS Building Footprints':['North Carolina','South Carolina']}

print('Select Infrastructure Type, Source, and State of Interest Below:')


def print_State(State):
    print (State)

def select_State(Source):
    StateW.options = geo[Source]


scW = widgets.Select(options=geo.keys())
init = scW.value
StateW = widgets.Select(options=geo[init])
j = widgets.interactive(print_State, State=StateW)
i = widgets.interactive(select_State, Source=scW)
display(i)
display(j)


print('Draw your coastal bounds region of interest in your chosen State within next cell. Once drawing is complete, run the cell after to download your file. This may take a while as source files are large.')

Select Infrastructure Type, Source, and State of Interest Below:


interactive(children=(Select(description='Source', options=('Open Street Maps', 'MS Building Footprints'), val…

interactive(children=(Select(description='State', options=('North Carolina', 'South Carolina'), value='North C…

Draw your coastal bounds region of interest in your chosen State within next cell. Once drawing is complete, run the cell after to download your file. This may take a while as source files are large.


In [9]:
if StateW.value=='North Carolina':

    #ROI name
    roi="NorthCarolina"
    #Set buffer distance in decimal degreees
    dist=0.01


    watercolor = basemap_to_tiles(basemaps.Esri.WorldImagery, crs='espg:4326')

    m = Map(layers=(watercolor, ), center=(35,-75 ), zoom=7.7)


    dc = DrawControl(marker={'shapeOptions': {'color': '#0000FF'}},
                     rectangle={'shapeOptions': {'color': '#0000FF'}},
                     circle={'shapeOptions': {'color': '#0000FF'}},
                     circlemarker={},
                     )
    def handle_draw(target, action, geo_json):
        print(action)
        print(geo_json)

    dc.on_draw(handle_draw)
    m.add_control(dc)
    
if StateW.value=='South Carolina':
       #ROI name
    roi="SouthCarolina"
    osm_roi='South Carolina, USA'
    #Set buffer distance in decimal degreees
    dist=0.01


    watercolor = basemap_to_tiles(basemaps.Esri.WorldImagery, crs='espg:4326')

    m = Map(layers=(watercolor, ), center=(33,-79 ), zoom=7.7)


    dc = DrawControl(marker={'shapeOptions': {'color': '#0000FF'}},
                     rectangle={'shapeOptions': {'color': '#0000FF'}},
                     circle={'shapeOptions': {'color': '#0000FF'}},
                     circlemarker={},
                     )
    def handle_draw(target, action, geo_json):
        print(action)
        print(geo_json)

    dc.on_draw(handle_draw)
    m.add_control(dc)
    
    
m

Map(center=[35, -75], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

created
{'type': 'Feature', 'properties': {'style': {'stroke': True, 'color': '#0000FF', 'weight': 4, 'opacity': 0.5, 'fill': True, 'fillColor': None, 'fillOpacity': 0.2, 'clickable': True}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-75.954157, 35.072192], [-75.954157, 35.175529], [-75.833282, 35.175529], [-75.833282, 35.072192], [-75.954157, 35.072192]]]}}


In [12]:
if scW.value == 'MS Building Footprints':

    bnds= (dc.last_draw).get("geometry").get("coordinates")[0]
    bnds_poly=Polygon(bnds)
    bounds_gpd=gpd.GeoDataFrame(geometry=[bnds_poly])
    bounds_gpd=bounds_gpd.set_crs('EPSG:4326')

    print("Bounds shape created from drawing")

    coast=gpd.read_file('https://geodata.lib.berkeley.edu/download/file/stanford-xv279yj9196-geojson.json')

    print('US coastline saved')


    #clip coastal polyline
    roi_coast=gpd.clip(coast, bounds_gpd)
    roi_coast=roi_coast.to_crs('EPSG:4326')
    buffer=roi_coast.buffer(dist)

    print('US Coastline buffered by 10km and clipped to bounds area')


    foot_url='https://usbuildingdata.blob.core.windows.net/usbuildings-v2/'+roi+'.geojson.zip'
    footfile= urlopen(foot_url).read()
    footfile=BytesIO(footfile)
    foot_doc=ZipFile(footfile)
    foot_cont=foot_doc.open(roi+'.geojson')
    foot_gpd=gpd.read_file(foot_cont)

    print('MS data saved')

    json_clipped = gpd.clip(foot_gpd, buffer)
    json_clipped.to_file(out+roi+'Bldgs.geojson', driver='GeoJSON')

    print('MS data clipped to coastal buffer roi and downloaded to folder')
    
if scW.value == 'Open Street Maps':
    
    bnds= (dc.last_draw).get("geometry").get("coordinates")[0]
    bnds_poly=Polygon(bnds)
    bounds_gpd=gpd.GeoDataFrame(geometry=[bnds_poly])
    bounds_gpd=bounds_gpd.set_crs('EPSG:4326')

    print("Bounds shape created from drawing")

    coast=gpd.read_file('https://geodata.lib.berkeley.edu/download/file/stanford-xv279yj9196-geojson.json')

    print('US coastline saved')
    #clip coastal polyline
    roi_coast=gpd.clip(coast, bounds_gpd)
    roi_coast=roi_coast.to_crs('EPSG:4326')
    buffer=roi_coast.buffer(dist)
    print('US Coastline buffered by 10km and clipped to bounds area')


    tags = {"building": True}
    north=bnds[1]
    north=north[1]
    south=bnds[0]
    south=south[1]
    west=bnds[0]
    west=west[0]
    east=bnds[2]
    east=east[0]

    gdf = ox.geometries_from_bbox(north, south, east, west, tags)
    gdf_proj = ox.project_gdf(gdf)
    # save as a geojson to your folder. Coordinate system set to EPSG 4326
    osmgdf_save = gdf.applymap(lambda x: str(x) if isinstance(x, list) else x)
    osmgdf_save.drop(labels="nodes", axis=1)
    osmgdf_save=osmgdf_save.drop(labels="nodes", axis=1)
    osm_gpd=gpd.GeoDataFrame(osmgdf_save)
    json_clipped = gpd.clip(osm_gpd, buffer)
    json_clipped.to_file(out+'/'+roi+'Bldgs.geojson', driver="GPKG")


    print('OSM Building data clipped to coastal buffer roi and downloaded to folder')

/home/will/anaconda3/envs/MainImage/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Bounds shape created from drawing
US coastline saved


<ipython-input-12-d00b05b9f964>:52: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buffer=roi_coast.buffer(dist)


US Coastline buffered by 10km and clipped to bounds area
OSM Building data clipped to coastal buffer roi and downloaded to folder


<ipython-input-12-d00b05b9f964>:73: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  json_clipped = gpd.clip(osm_gpd, buffer)
